# Topological Index Calculator
This notebook calculates different topological indices (Edge Density, Wiener Index, Petitjean Index) from molecular structures.

In [1]:
import os
import sys


In [2]:
def get_index_choice():
    while True:
        print('Which topological index would you like to calculate?')
        print('1 - Edge Density')
        print('2 - Wiener Index')
        print('3 - Petitjean Index')
        print('4 - All of the above')

        choice = input('Enter your choice (1-4): ').strip()

        options = {
            '1': 'edge',
            '2': 'wiener',
            '3': 'petitjean',
            '4': 'all'
        }

        selected = options.get(choice)
        if selected:
            return selected
        print('Invalid choice. Please select 1, 2, 3, or 4.')


In [3]:
def get_input_path():
    input_path = input('Enter the path to a .mol file, a folder of .mol files, or a .sdf file: ').strip()

    if not os.path.exists(input_path):
        print('The specified path does not exist.')
        sys.exit(1)

    return input_path


In [4]:
def detect_input_type(input_path):
    if os.path.isdir(input_path):
        return 'mol_folder'
    elif input_path.endswith('.mol'):
        return 'mol_file'
    elif input_path.endswith('.sdf'):
        return 'sdf_file'
    else:
        print('Unsupported file type. Please provide a .mol file, folder, or .sdf file.')
        sys.exit(1)


In [5]:
# Example usage:
index_choice = get_index_choice()
input_path = get_input_path()
input_type = detect_input_type(input_path)

print(f'Selected index: {index_choice}')
print(f'Input path: {input_path}')
print(f'Input type: {input_type}')